<a href="https://colab.research.google.com/github/HalyshAnton/Python-AI/blob/AI_4_practice/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%BD%D0%B0_%D1%80%D0%B5%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%B8%D0%B7%D0%B0%D1%86%D1%96%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпорт модулів

In [1]:
import pandas as pd
import torch

## Стовпчики:

Ця таблиця містить інформацію про клієнтів банку та їхні характеристики, які можуть впливати на рішення щодо надання кредиту.

- **age** – Вік клієнта (ціле число).
- **is_working** – Ознака того, чи працює клієнт
- **job** – Тип зайнятості клієнта
- **marital** – Сімейний стан
- **education** – Рівень освіти
- **balance** – Баланс клієнта на банківському рахунку
- **housing** – Чи має клієнт іпотеку
- **loan** – Чи має клієнт активний кредит
- **contact** – Спосіб контакту з клієнтом
- **campaign** – Кількість контактів клієнта до банку
- **poutcome** – Результат попередньої маркетингової кампанії
- **offer** – Чи було клієнту запропоновано кредит
- **credit** – Сума кредиту

In [18]:
# трохи обробки даних
df = pd.read_csv('https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/refs/heads/main/module3/data/bank.csv')

df.loc[df['offer']=='no', 'credit'] = 0


In [19]:
df.head()

,age,is_working,job,marital,education,balance,housing,loan,contact,campaign,poutcome,offer,credit
0,29,no,technician,single,tertiary,18254,no,no,cellular,2,unknown,no,0
1,26,no,services,single,secondary,512,yes,yes,unknown,3,unknown,no,0
2,30,no,management,single,secondary,135,no,no,cellular,2,unknown,no,0
3,41,no,technician,married,unknown,30,yes,no,cellular,1,unknown,no,0
4,27,no,admin.,single,secondary,321,no,yes,unknown,1,unknown,no,0


Ваша задача побудувати модель, яка використовує класифікацію для визначення чи варто пропонувати кредит клієнту, та регресію для визначення суми кредиту

# Завдання 1
Розділіть дані на 3 частини: `offer`, `credit` та решту стовпчиків(дані клієнта)


Виведіть кількості категорій по кожному категоріальному стовпчику, на їх основі визначіть ті категорії яких дуже мало

Змініть препроцесор:
* до обробки числових даних добавте `Quantiletransformer`, це потрібно шоб всі значення були в діаразоні від 0 до 1
* до категоріальної обробки добавте `OneHotEncoder`

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, QuantileTransformer

num_columns = X.select_dtypes(include="number").columns
cat_columns = X.select_dtypes(include="object").columns

# числові ознаки
num_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median"))]
    )

# категоріальні ознаки
cat_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent")),
           ("encoder", OrdinalEncoder())
           ])

# об'єднання двох pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_columns),
        ("cat", cat_transformer, cat_columns),
    ]
)

preprocessor.set_output(transform='pandas')
preprocessor

# Завдання 2
Підготуйте датасети

Для класифікації візміть усі рядки з таблиці та створіть `dataloader`

In [ ]:
from torch.utils.data import Dataset, Dataloader

class MyDataset(Dataset):
    def __init__(self, X, y):
        super().__init__()
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)
        self.y = self.y.reshape(-1, 1)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, ind):
        return self.X[ind], self.y[ind]



In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8,
                                                    random_state=42
                                                    )

In [ ]:
batch_size = 128
train_dl = DataLoader(dataset_train,
                      batch_size=batch_size,
                      shuffle=True,
                      )

test_dl = DataLoader(dataset_test,
                     batch_size=batch_size,
                     shuffle=True,
                     )

Для регресії візьміть дані тих клієнтів, яким запропонували кредит

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8,
                                                    random_state=42
                                                    )

In [ ]:
batch_size = 128
train_dl = DataLoader(dataset_train,
                      batch_size=batch_size,
                      shuffle=True,
                      )

test_dl = DataLoader(dataset_test,
                     batch_size=batch_size,
                     shuffle=True,
                     )

# Завдання 3
Створіть 2 нейромережі для класифікації та регресії

# Завдання 4
Визначте функцію втрат та оптимізатор для обох моделей

In [ ]:
loss_fn = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Завдання 5
Натренуйте обидві нейромережі

In [ ]:
# @title Функція для тренування
import time

def train(model, optimizer, loss_fn, train_dl, val_dl,
          metrics=None, metrics_name=None, epochs=20, device='cpu', task='regression'):
    '''
    Runs training loop for classification problems. Returns Keras-style
    per-epoch history of loss and accuracy over training and validation data.

    Parameters
    ----------
    model : nn.Module
        Neural network model
    optimizer : torch.optim.Optimizer
        Search space optimizer (e.g. Adam)
    loss_fn :
        Loss function (e.g. nn.CrossEntropyLoss())
    train_dl :
        Iterable dataloader for training data.
    val_dl :
        Iterable dataloader for validation data.
    metrics: list
        List of sklearn metrics functions to be calculated
    metrics_name: list
        List of matrics names
    epochs : int
        Number of epochs to run
    device : string
        Specifies 'cuda' or 'cpu'
    task : string
        type of problem. It can be regression, binary or multiclass

    Returns
    -------
    Dictionary
        Similar to Keras' fit(), the output dictionary contains per-epoch
        history of training loss, training accuracy, validation loss, and
        validation accuracy.
    '''

    print('train() called: model=%s, opt=%s(lr=%f), epochs=%d, device=%s\n' % \
          (type(model).__name__, type(optimizer).__name__,
           optimizer.param_groups[0]['lr'], epochs, device))

    metrics = metrics if metrics else []
    metrics_name = metrics_name if metrics_name else [metric.__name__ for metric in metrics]

    history = {} # Collects per-epoch loss and metrics like Keras' fit().
    history['loss'] = []
    history['val_loss'] = []
    for name in metrics_name:
        history[name] = []
        history[f'val_{name}'] = []

    start_time_train = time.time()

    for epoch in range(epochs):

        # --- TRAIN AND EVALUATE ON TRAINING SET -----------------------------
        start_time_epoch = time.time()

        model.train()
        history_train = {name: 0 for name in ['loss']+metrics_name}

        for batch in train_dl:
            x    = batch[0].to(device)
            y    = batch[1].to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            y_pred = y_pred.detach().cpu().numpy()
            y = y.detach().cpu().numpy()


            history_train['loss'] += loss.item() * x.size(0)
            for name, func in zip(metrics_name, metrics):
              try:
                  history_train[name] += func(y, y_pred) * x.size(0)
              except:
                  if task == 'binary': y_pred_ = y_pred.round()
                  elif task == 'multiclass': y_pred_ = y_pred.argmax(axis=-1)
                  history_train[name] += func(y, y_pred_) * x.size(0)

        for name in history_train:
            history_train[name] /= len(train_dl.dataset)


        # --- EVALUATE ON VALIDATION SET -------------------------------------
        model.eval()
        history_val = {'val_' + name: 0 for name in metrics_name+['loss']}

        with torch.no_grad():
            for batch in val_dl:
                x    = batch[0].to(device)
                y    = batch[1].to(device)
                y_pred = model(x)
                loss = loss_fn(y_pred, y)

                y_pred = y_pred.cpu().numpy()
                y = y.cpu().numpy()

                history_val['val_loss'] += loss.item() * x.size(0)
                for name, func in zip(metrics_name, metrics):
                    try:
                        history_val['val_'+name] += func(y, y_pred) * x.size(0)
                    except:
                        if task == 'binary': y_pred_ = y_pred.round()
                        elif task == 'multiclass': y_pred_ = y_pred.argmax(axis=-1)

                        history_val['val_'+name] += func(y, y_pred_) * x.size(0)

        for name in history_val:
            history_val[name] /= len(val_dl.dataset)

        # PRINTING RESULTS

        end_time_epoch = time.time()

        for name in history_train:
            history[name].append(history_train[name])
            history['val_'+name].append(history_val['val_'+name])

        total_time_epoch = end_time_epoch - start_time_epoch

        print(f'Epoch {epoch+1:4d} {total_time_epoch:4.0f}sec', end='\t')
        for name in history_train:
            print(f'{name}: {history[name][-1]:10.3g}', end='\t')
            print(f"val_{name}: {history['val_'+name][-1]:10.3g}", end='\t')
        print()

    # END OF TRAINING LOOP

    end_time_train       = time.time()
    total_time_train     = end_time_train - start_time_train
    print()
    print('Time total:     %5.2f sec' % (total_time_train))

    return history

In [ ]:
import matplotlib.pyplot as plt

def plot_metric(history, name):
  plt.title(f"Model results with {name}")
  plt.plot(history[name], label='train')
  plt.plot(history['val_'+name], label='val')
  plt.xlabel('Epoch')
  plt.ylabel(name)
  plt.legend()


In [ ]:
from sklearn import metrics


def get_metrics(y_true, y_pred, name="model"):
  df = pd.DataFrame()

  df.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
  df.loc["RMSE", name] = metrics.mean_squared_error(y_true, y_pred) ** 0.5
  df.loc["R2", name] = metrics.r2_score(y_true, y_pred)

  return df.round(2)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

y_pred = model.predict(X_test)

y_pred = (y_pred > 0.5)

ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

# Завдання 6
Збережіть препроцесор та моделі в файли

Напишіть клас CreditModel
* init -- підвантажує моделі та препроцесор
* predict -- робить прогноз, якщо класифікатор каже що можна давати кредит, прогнозує суму кредиту. Повертає 2 значення
    * offer -- True, якщо кредит пропонується
    * credit -- сума кредиту, None, якщо не пропонується